### 1.0 import libraries

In [142]:
import os
import random

import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg

### 1.1 load macrovoxels

In [143]:
# load lattice CSV file
lattice_path = os.path.relpath('../data/macrovoxels.csv')
macro_lattice = tg.lattice_from_csv(lattice_path)

### 1.x generate a single random configuration to be tested

In [144]:
# create random configuration
#config_rand = [random.randint(0,1) for i in range(125)]
config_rand = [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0]

config_rand = np.reshape(config_rand,(5,5,5))

In [145]:
# saving the random configuration for future use
random_lattice = tg.to_lattice(config_rand, macro_lattice)
save_path = os.path.relpath("../data/random_lattice.csv")
random_lattice.to_csv(save_path)

## Identifying the roof

In [146]:
# shifting all the voxels one level sideways --> -Y direction
#shifted_lattice_Y_neg = np.roll(random_lattice_padded, (0,0,-1),axis=1) 

In [147]:
# shifting all the voxels one level sideways --> +X direction
#shifted_lattice_X_pos = np.roll(random_lattice_padded, (0,0,1),axis=2) 

In [148]:
# shifting all the voxels one level sideways --> -X direction
#shifted_lattice_X_neg = np.roll(random_lattice_padded, (0,0,-1),axis=2) 

In [149]:
# # padding to avoid the rolling issue
# random_lattice_padded = np.pad(random_lattice, 1, mode='constant',constant_values=0)

# # shifting all the voxels one level sideways --> +Y direction
# # TODO: unify into single function for each meshing approach from here to mesh list
# shifted_lattice_Y_pos = np.roll(random_lattice_padded, (0,0,1),axis=1) 

# # an exposed facade surface exists where a voxel is filled (1) and the voxel next to it is empty (0)
# side_voxels_3darray_padded = (random_lattice_padded == 1) *  (shifted_lattice_Y_pos == 0)

# # removing the pad
# side_voxels_3darray = side_voxels_3darray_padded[1:-1, 1:-1, 1:-1]
# # convert to lattice
# side_voxels_lattice = tg.to_lattice(side_voxels_3darray, random_lattice)

# # extracting the centroids of all side voxels
# side_centroids = side_voxels_lattice.centroids

In [150]:
# def construct_vertical_mesh(lat):
#     vertical_meshes = []

#     # padding to avoid the rolling issue
#     input_lattice_padded = np.pad(lat, 1, mode='constant',constant_values=0)

#     # Y_positive
#     Y_pos_centroids = find_centroids(lattice= input_lattice_padded, dir= 1, axes= 1)
#     Y_pos_mesh = construct_mesh_y_pos(Y_pos_centroids)
#     vertical_meshes.extend(Y_pos_mesh)

#     # Y_negative
#     Y_neg_centroids = find_centroids(lattice= input_lattice_padded, dir= -1, axes= 1)
#     Y_neg_mesh = construct_mesh_y_neg(Y_neg_centroids)
#     vertical_meshes.extend(Y_neg_mesh)

#     # X_positive
#     X_pos_centroids = find_centroids(lattice= input_lattice_padded, dir= 1, axes= 0)
#     X_pos_mesh = construct_mesh_x_pos(X_pos_centroids)
#     vertical_meshes.extend(X_pos_mesh)

#     # X_negative
#     X_neg_centroids = find_centroids(lattice= input_lattice_padded, dir= -1, axes= 0)
#     X_neg_mesh = construct_mesh_x_neg(X_neg_centroids)
#     vertical_meshes.extend(X_neg_mesh)
    
#     return vertical_meshes

In [151]:
def find_centroids(lattice, ref_lattice, dir, axes):

    # shifting all the voxels one level in a certain direction
    shifted_lattice_Y_pos = np.roll(lattice, (0,0,dir),axis=axes) 
    
    # an exposed facade surface exists where a voxel is filled (1) and the voxel next to it is empty (0)
    side_voxels_3darray_padded = (lattice == 1) *  (shifted_lattice_Y_pos == 0)

    # removing the pad
    side_voxels_3darray = side_voxels_3darray_padded[1:-1, 1:-1, 1:-1]
    
    # convert to lattice
    side_voxels_lattice = tg.to_lattice(side_voxels_3darray, ref_lattice)

    # extracting the centroids of all exposed voxels
    centroids = side_voxels_lattice.centroids

    return centroids

In [152]:
# Function for creating meshes

def construct_mesh_y_pos(centroid, unit):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ 1, -1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([ 1,-1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([-1,-1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([-1, -1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[0,2,1]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[0,2,1]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [153]:
# Function for creating meshes

def construct_mesh_y_neg(centroid, unit):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ 1, 1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([ 1,1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([-1,1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([-1, 1, 1]) # side left above

        face_a = [v1,v2,v3] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v3,v0,v1]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[1,2,0]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[1,2,0]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [154]:
# Function for creating meshes

def construct_mesh_x_pos(centroid, unit):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ -1, -1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([ -1,-1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([-1,1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([-1, 1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[2,1,0]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[2,1,0]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [155]:
# Function for creating meshes

def construct_mesh_x_neg(centroid, unit):
    meshes= []
    for i, cen in enumerate(centroid):
        # generating the vertices of the side faces in +Y direction
        # centroid + half of the unit size in the four top directions
        v0 = cen + 0.5 * unit * np.array([ 1, 1, 1]) # side right above
        v1 = cen + 0.5 * unit * np.array([1,1, -1]) # side right below
        v2 = cen + 0.5 * unit * np.array([1,-1, -1]) # side left below
        v3 = cen + 0.5 * unit * np.array([1, -1, 1]) # side left above

        face_a = [v0,v1,v2] # trimesh only takes triangular meshes, no quad meshes
        face_b = [v2,v3,v0]

        mesh_a = tm.Trimesh(vertices= face_a, faces= [[2,1,0]])
        mesh_b = tm.Trimesh(vertices= face_b, faces= [[2,1,0]])

        meshes.append(mesh_a)
        meshes.append(mesh_b)

    return meshes

In [156]:
# this function could be much shorter probably --> 4x almost identical operations, does works fine

def construct_vertical_mesh(lat,unit):
    vertical_meshes = []
    test_points = []
    test_point_normals = []

    # padding to avoid the rolling issue
    input_lattice_padded = np.pad(lat, 1, mode='constant',constant_values=0)

    # Y_positive mesh
    Y_pos_centroids = find_centroids(lattice= input_lattice_padded, ref_lattice=lat, dir= 1, axes= 1)
    Y_pos_mesh = construct_mesh_y_pos(Y_pos_centroids, unit)
    vertical_meshes.extend(Y_pos_mesh)

    # move the test points so they are not inside mesh edge
    Y_pos_text_point = Y_pos_centroids + (lat.unit/2 + 0.01) * [0,-1,0]
    test_points.extend(Y_pos_text_point)

    # find normal of squares: take every other triangle normal
    Y_pos_mesh = tm.util.concatenate(Y_pos_mesh)
    Y_pos_normals = Y_pos_mesh.face_normals[::2]
    test_point_normals.extend(Y_pos_normals)
    
    # Y_negative mesh
    Y_neg_centroids = find_centroids(lattice= input_lattice_padded, ref_lattice=lat, dir= -1, axes= 1)
    Y_neg_mesh = construct_mesh_y_neg(Y_neg_centroids, unit)
    vertical_meshes.extend(Y_neg_mesh)

    # move the test points so they are not inside mesh edge
    Y_neg_text_point = Y_neg_centroids + (lat.unit/2 + 0.01) * [0,1,0]
    test_points.extend(Y_neg_text_point)

    # find normal of squares: take every other triangle normal
    Y_neg_mesh = tm.util.concatenate(Y_neg_mesh)
    Y_neg_normals = Y_neg_mesh.face_normals[::2]
    test_point_normals.extend(Y_neg_normals)

    # X_positive mesh
    X_pos_centroids = find_centroids(lattice= input_lattice_padded, ref_lattice=lat, dir= 1, axes= 0)
    X_pos_mesh = construct_mesh_x_pos(X_pos_centroids, unit)
    vertical_meshes.extend(X_pos_mesh)

    # move the test points so they are not inside mesh edge
    X_pos_text_point = X_pos_centroids + (lat.unit/2 + 0.01) * [-1,0,0]
    test_points.extend(X_pos_text_point)

    # find normal of squares: take every other triangle normal
    X_pos_mesh = tm.util.concatenate(X_pos_mesh)
    X_pos_normals = X_pos_mesh.face_normals[::2]
    test_point_normals.extend(X_pos_normals)

    # X_negative mesh
    X_neg_centroids = find_centroids(lattice= input_lattice_padded, ref_lattice=lat, dir= -1, axes= 0)
    X_neg_mesh = construct_mesh_x_neg(X_neg_centroids, unit)
    vertical_meshes.extend(X_neg_mesh)

    # move the test points so they are not inside mesh edge
    X_neg_text_point = X_neg_centroids + (lat.unit/2 + 0.01) * [1,0,0]
    test_points.extend(X_neg_text_point)

    # find normal of squares: take every other triangle normal
    X_neg_mesh = tm.util.concatenate(X_neg_mesh)
    X_neg_normals = X_neg_mesh.face_normals[::2]
    test_point_normals.extend(X_neg_normals)

    return vertical_meshes, test_points, test_point_normals

In [157]:
unit = random_lattice.unit

In [158]:
facade_mesh, test_points, test_point_normals = construct_vertical_mesh(random_lattice,unit)
facade_mesh = tm.util.concatenate(facade_mesh)

### Merge meshes

In [159]:
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

### 1.x visuzalizing

In [163]:
p = pv.Plotter(notebook=True)

p.add_mesh(tri_to_pv(facade_mesh), color='#abd8ff', opacity=1)
#random_lattice.fast_vis(p,False,False,opacity=1)
p.add_arrows(np.array(test_points),np.array(test_point_normals),mag=5)
p.add_points(np.array(test_points))

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(218.05727316311544, 98.0572712557668, 158.05727220944112),
 (60.000000953674316, -60.000000953674316, 0.0),
 (0.0, 0.0, 1.0)]